
# ODE Notebook — Exponential Growth and Lotka–Volterra

This notebook refactors and completes your code into two clean sections:

- **A. Exponential test** \(y' = y\) with vector initial conditions — exact solution vs `solve_ivp` (RK45), **Euler**, and **RK4**, plus error plots.
- **B. Lotka–Volterra** predator–prey model — time series, **phase portrait**, vector field with **nullclines**, and conservation-law check.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Keep plots simple and readable
# (No external styles to keep dependencies minimal)


## A) Exponential test: $y'=y$ (vector y)

In [ ]:

def f_exp(t, y):
    return y

def exact_exp(t, y0):
    # y0 is a vector; broadcast exp(t)
    return np.exp(t) * y0

# Integrate with solve_ivp (RK45)
T = 1.0
t_eval = np.linspace(0.0, T, 50)
y0_vec = np.array([1.0, 2.0, 3.0])
sol = solve_ivp(f_exp, (0.0, T), y0_vec, method="RK45", t_eval=t_eval)

# Plot time series
plt.figure()
for i in range(len(y0_vec)):
    plt.plot(sol.t, sol.y[i], label=f"$y_{i+1}(t)$")
plt.xlabel("t"); plt.title("RK45 solution for $y'=y$ (vector ICs)"); plt.legend(); plt.show()

print("t grid:", sol.t)
print("y shape:", sol.y.shape)


In [ ]:

def euler_explicit(f, y0, N, T):
    t = np.linspace(0.0, T, N+1)
    h = t[1] - t[0]
    y = np.zeros((N+1, len(y0)), dtype=float)
    y[0] = y0
    for n in range(N):
        y[n+1] = y[n] + h * f(t[n], y[n])
    return t, y

def rk4(f, y0, N, T):
    t = np.linspace(0.0, T, N+1)
    h = t[1] - t[0]
    y = np.zeros((N+1, len(y0)), dtype=float)
    y[0] = y0
    for n in range(N):
        tn, yn = t[n], y[n]
        k1 = f(tn, yn)
        k2 = f(tn + 0.5*h, yn + 0.5*h*k1)
        k3 = f(tn + 0.5*h, yn + 0.5*h*k2)
        k4 = f(tn + h,       yn + h*k3)
        y[n+1] = yn + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    return t, y

# Compare Euler and RK4 with exact solution at T
for N in (10, 20, 40, 80):
    tE, yE = euler_explicit(f_exp, y0_vec, N, T)
    tR, yR = rk4(f_exp, y0_vec, N, T)
    y_exact_T = exact_exp(T, y0_vec)

    eE = np.linalg.norm(yE[-1] - y_exact_T, ord=2)
    eR = np.linalg.norm(yR[-1] - y_exact_T, ord=2)
    print(f"N={N:3d}  ||Euler(T)-exact||_2 = {eE:.3e}   ||RK4(T)-exact||_2 = {eR:.3e}")

# Plot solutions for a representative N
N = 40
tE, yE = euler_explicit(f_exp, y0_vec, N, T)
tR, yR = rk4(f_exp, y0_vec, N, T)
t_ref = np.linspace(0.0, T, 400)
Yref = np.vstack([exact_exp(t, y0_vec) for t in t_ref]).T  # shape (3, len(t_ref))

plt.figure()
for i in range(len(y0_vec)):
    plt.plot(tE, yE[:,i], '+', label=f"Euler y{i+1}")
    plt.plot(tR, yR[:,i], 'x', label=f"RK4 y{i+1}")
    plt.plot(t_ref, Yref[i], label=f"Exact y{i+1}")
plt.xlabel("t"); plt.title("y'=y — Euler vs RK4 vs exact"); plt.legend(); plt.show()


## B) Lotka–Volterra predator–prey

In [ ]:

# Parameters (your values)
alpha = 0.25   # prey growth
beta  = -0.01  # prey loss due to predation
gamma = -1.0   # predator death
delta = 0.01   # predator growth due to predation

def lotka_v(t, y):
    x, z = y  # x=prey, z=predator
    return np.array([ alpha*x + beta*x*z,
                      gamma*z + delta*x*z ])

# Initial conditions and integration
Tlv = 20.0
t_eval_lv = np.linspace(0.0, Tlv, 600)
y0_lv = np.array([80.0, 30.0])

sol_lv = solve_ivp(lotka_v, (0.0, Tlv), y0_lv, method="RK45", t_eval=t_eval_lv)

# Time series
plt.figure()
plt.plot(sol_lv.t, sol_lv.y[0], label="Prey x(t)")
plt.plot(sol_lv.t, sol_lv.y[1], label="Predator y(t)")
plt.xlabel("t"); plt.title("Lotka–Volterra time series"); plt.legend(); plt.show()

# Phase portrait (trajectory)
plt.figure()
plt.plot(sol_lv.y[0], sol_lv.y[1])
plt.xlabel("Prey x"); plt.ylabel("Predator y"); plt.title("Phase portrait (trajectory)"); plt.show()

# Vector field + nullclines
xmin, xmax = 70, 140
ymin, ymax = 10, 50
x = np.linspace(xmin, xmax, 20)
y = np.linspace(ymin, ymax, 20)
X, Y = np.meshgrid(x, y)
U = alpha*X + beta*X*Y
V = gamma*Y + delta*X*Y

plt.figure()
plt.quiver(X, Y, U, V)
# Nullclines: xdot=0 => alpha + beta y = 0 => y = -alpha/beta
#             ydot=0 => gamma + delta x = 0 => x = -gamma/delta
y_nc = -alpha/beta
x_nc = -gamma/delta
plt.axhline(y_nc, linestyle='--', label=r'$\dot x=0$')
plt.axvline(x_nc, linestyle='--', label=r'$\dot y=0$')
plt.xlim(xmin, xmax); plt.ylim(ymin, ymax)
plt.xlabel("Prey x"); plt.ylabel("Predator y")
plt.title("Vector field with nullclines")
plt.legend(); plt.show()

print("Nullclines at:  y* = -alpha/beta =", y_nc, "   x* = -gamma/delta =", x_nc)


### Conservation-law check

In [ ]:

# Classic LV invariant (for this parametrization):
# H(x,y) = delta x - gamma ln x + beta y - alpha ln y  (constant along solutions)
def H_lv(x, y):
    return delta*x - gamma*np.log(x) + beta*y - alpha*np.log(y)

H_vals = H_lv(sol_lv.y[0], sol_lv.y[1])
plt.figure()
plt.plot(sol_lv.t, H_vals - H_vals[0])
plt.xlabel("t"); plt.ylabel("H(t) - H(0)")
plt.title("Lotka–Volterra invariant drift (numerical)")
plt.show()
print("Invariant drift (min, max):", np.min(H_vals - H_vals[0]), np.max(H_vals - H_vals[0]))
